In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
import string
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.models import Model
from sklearn.metrics import precision_recall_fscore_support



from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM,Dropout , Input , Bidirectional , concatenate
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
#from tensorflow.keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.optimizers import SGD
import seaborn as sns
from numpy import array
from numpy import asarray
from numpy import zeros
from sklearn.base import BaseEstimator, TransformerMixin
import emoji



In [2]:
twitterdata = pd.read_csv("dataPandas.csv")
twitterdata.isnull().values.any()
twitterdata.shape

(5000, 2)

In [3]:
class TextCounts(BaseEstimator, TransformerMixin):
    
    def count_regex(self, pattern, tweet):
        return len(re.findall(pattern, tweet))
    
    def fit(self, X, y=None, **fit_params):
        # fit method is used when specific operations need to be done on the train data, but not on the test data
        return self
    
    def transform(self, X, **transform_params):
        count_words = X.apply(lambda x: self.count_regex(r'\w+', x)) 
        count_mentions = X.apply(lambda x: self.count_regex(r'@\w+', x))
        count_hashtags = X.apply(lambda x: self.count_regex(r'#\w+', x))
        count_capital_words = X.apply(lambda x: self.count_regex(r'\b[A-Z]{2,}\b', x))
        count_excl_quest_marks = X.apply(lambda x: self.count_regex(r'!|\?', x))
        count_urls = X.apply(lambda x: self.count_regex(r'http.?://[^\s]+[\s]?', x))
        # We will replace the emoji symbols with a description, which makes using a regex for counting easier
        # Moreover, it will result in having more words in the tweet
        count_emojis = X.apply(lambda x: emoji.demojize(x)).apply(lambda x: self.count_regex(r':[a-z_&]+:', x))
        
        df = pd.DataFrame({'count_words': count_words
                           , 'count_mentions': count_mentions
                           , 'count_hashtags': count_hashtags
                           , 'count_capital_words': count_capital_words
                           , 'count_excl_quest_marks': count_excl_quest_marks
                           , 'count_urls': count_urls
                           , 'count_emojis': count_emojis
                          })
        
        return df
    
tc = TextCounts()
twitterdata_eda = tc.fit_transform(twitterdata.Tweets)
#twitterdata_eda['Labels'] = twitterdata.Labels

In [4]:
twitterdata_meta = twitterdata_eda.to_numpy()
twitterdata_meta.shape

(5000, 7)

In [5]:
class CleanText(BaseEstimator, TransformerMixin):
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)    
   
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords).apply(self.stemming)
        return clean_X

In [6]:
ct = CleanText()
twitterdata_CT = ct.fit_transform(twitterdata.Tweets)
#twitterdata_CT.head()
twitterdata['cTweets'] = twitterdata_CT

In [7]:
X = []
sentences = list(twitterdata['cTweets'])
for sen in sentences:
    #X.append(preprocess_text(sen))
    X.append(sen)

In [8]:
y = twitterdata['Labels']

y = np.array(list(map(lambda x: 1 if x=="SARCASM" else 0, y)))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(twitterdata_eda, y, test_size=0.20, random_state=42)

In [10]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [11]:
embeddings_dictionary = dict()
#glove_file = open('./Data/glove.twitter.27B.100d.txt', encoding="utf8")
glove_file = open('./Data/glove.6B.100d.txt', encoding="utf8")


for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [12]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [27]:
nlp_input1 = Input(shape=(maxlen,)) 
meta_input1 = Input(shape=(7,))
#emb = Embedding(output_dim=embedding_size, input_dim=100, input_length=seq_length)
emb1 = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)(nlp_input1) 
#lstm = LSTM(300, dropout=0.3, recurrent_dropout=0.3)(embed)
#nlp_out2 = Bidirectional(LSTM(128))(meta_input1) 
x = Dense(8, activation="relu")(meta_input1)


nlp_out1 = Bidirectional(LSTM(128))(emb1) 
concat1 = concatenate([nlp_out1, x]) 
drop1 = Dropout(0.6)(concat1)
#dens = Dense(1)(drop)

#classifier = Dense(32, activation='relu')(drop) 
output1 = Dense(1, activation='sigmoid')(drop1) 
model_blstm1 = Model(inputs=[nlp_input1 , meta_input1], outputs=[output1])

model_blstm1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_blstm1.summary())

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 100, 100)     1968800     input_13[0][0]                   
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 7)]          0                                            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 256)          234496      embedding_6[0][0]                
_______________________________________________________________________________________

In [28]:
history = model_blstm1.fit([X_train,X_train_meta ], y_train, batch_size=128, epochs=30, verbose=1, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 7s 292ms/step - loss: 2.5480 - acc: 0.5141 - val_loss: 0.9496 - val_acc: 0.6225
Epoch 2/30
25/25 [==============================] - 6s 252ms/step - loss: 1.8139 - acc: 0.5581 - val_loss: 0.8001 - val_acc: 0.6050
Epoch 3/30
25/25 [==============================] - 7s 261ms/step - loss: 1.2791 - acc: 0.5875 - val_loss: 0.6997 - val_acc: 0.6725
Epoch 4/30
25/25 [==============================] - 7s 260ms/step - loss: 0.9769 - acc: 0.6372 - val_loss: 0.6280 - val_acc: 0.7025
Epoch 5/30
25/25 [==============================] - 6s 257ms/step - loss: 0.7589 - acc: 0.6572 - val_loss: 0.5868 - val_acc: 0.6988
Epoch 6/30
25/25 [==============================] - 6s 251ms/step - loss: 0.6603 - acc: 0.6716 - val_loss: 0.5651 - val_acc: 0.6913
Epoch 7/30
25/25 [==============================] - 6s 252ms/step - loss: 0.5984 - acc: 0.6963 - val_loss: 0.5545 - val_acc: 0.6913
Epoch 8/30
25/25 [==============================] - 7s 262ms/step - loss: 0.

In [13]:
nlp_input = Input(shape=(maxlen,)) 
meta_input = Input(shape=(7,))
#emb = Embedding(output_dim=embedding_size, input_dim=100, input_length=seq_length)
emb = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)(nlp_input) 
#lstm = LSTM(300, dropout=0.3, recurrent_dropout=0.3)(embed)

nlp_out = Bidirectional(LSTM(128 , dropout=0.3, recurrent_dropout=0.3))(emb) 
concat = concatenate([nlp_out, meta_input]) 
drop = Dropout(0.5)(concat)
#dens = Dense(1)(drop)

#classifier = Dense(32, activation='relu')(drop) 
output = Dense(1, activation='sigmoid')(drop) 
model_blstm = Model(inputs=[nlp_input , meta_input], outputs=[output])

model_blstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_blstm.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     1968800     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 256)          234496      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7)]          0                                            
_______________________________________________________________________________________

In [14]:
history = model_blstm.fit([X_train,X_train_meta ], y_train, batch_size=128, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
25/25 [==============================] - 23s 904ms/step - loss: 1.4659 - acc: 0.4997 - val_loss: 0.7012 - val_acc: 0.5612
Epoch 2/50
25/25 [==============================] - 23s 910ms/step - loss: 0.8647 - acc: 0.5647 - val_loss: 0.6256 - val_acc: 0.6550
Epoch 3/50
25/25 [==============================] - 23s 933ms/step - loss: 0.7346 - acc: 0.6228 - val_loss: 0.5492 - val_acc: 0.7025
Epoch 4/50
25/25 [==============================] - 24s 963ms/step - loss: 0.6589 - acc: 0.6700 - val_loss: 0.5242 - val_acc: 0.7188
Epoch 5/50
25/25 [==============================] - 25s 985ms/step - loss: 0.6121 - acc: 0.6759 - val_loss: 0.5237 - val_acc: 0.7287
Epoch 6/50
25/25 [==============================] - 25s 1s/step - loss: 0.5732 - acc: 0.7081 - val_loss: 0.5200 - val_acc: 0.7225
Epoch 7/50
25/25 [==============================] - 25s 998ms/step - loss: 0.5427 - acc: 0.7109 - val_loss: 0.5230 - val_acc: 0.7237
Epoch 8/50
25/25 [==============================] - 26s 1s/step - loss: 

In [15]:
score,accuracy = model_blstm.evaluate([X_test,X_test_meta], y_test, verbose=1)

32/32 [==============================] - 1s 22ms/step - loss: 0.9499 - acc: 0.6920


In [16]:
print("Test Score:", score)
print("Test Accuracy:", accuracy)

Test Score: 0.9499468803405762
Test Accuracy: 0.6919999718666077


In [ ]:
twittertestdata = pd.read_csv("dftestdata.csv")
print(twittertestdata.isnull().values.any())
twittertestdata_CT = ct.fit_transform(twittertestdata.Tweets)
twittertestdata_eda = tc.fit_transform(twittertestdata.Tweets)
twittertestdata_meta = twittertestdata_eda.to_numpy()


twittertestdata['cTweets'] = twittertestdata_CT

In [ ]:
X_val = []
sentences = list(twittertestdata['cTweets'])
for sen in sentences:
    X_val.append(sen)
    
X_validate = tokenizer.texts_to_sequences(X_val)
#X_valTokens = tokenizer.texts_to_sequences(X_validate)
X_valTokens = pad_sequences(X_validate, padding='post', maxlen=maxlen)

In [ ]:
validation = model_blstm.predict([X_valTokens,twittertestdata_meta])

In [ ]:
twittertestdata['Predict'] = validation

In [ ]:
twittertestdata['PLabel'] = np.where(twittertestdata['Predict'] > 0.5, "SARCASM", "NOT_SARCASM")

In [ ]:
twittertestdata.head()

In [ ]:
twittertestdata.to_csv('answer_fblstm1.txt', columns = ["ID" , "PLabel"] , header = False , index = False)